In [ ]:
%stop_session

# Setup


In [ ]:
%iam_role IAM_ROLE_ARN
%region us-east-1
%glue_version 4.0
%idle_timeout 120
%number_of_workers 2
%extra_jars "s3://crawler-public/json/serde/json-serde.jar"
%spark_conf spark.sql.catalog.catalog.warehouse=s3://bucket/iceberg --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO
%%configure
{
    "datalake-formats": "iceberg"
}



In [ ]:
import org.apache.spark.sql._